In [1]:
@file:DependsOn("ai.koog:koog-agents-jvm:0.5.0")
%use serialization

In [5]:
import ai.koog.prompt.executor.clients.anthropic.AnthropicLLMClient
import ai.koog.prompt.executor.llms.all.simpleAnthropicExecutor
import ai.koog.prompt.executor.ollama.client.OllamaClient
import ai.koog.prompt.executor.llms.all.simpleOllamaAIExecutor
import ai.koog.prompt.llm.LLMCapability
import ai.koog.prompt.llm.LLMProvider
import ai.koog.prompt.llm.LLModel

val ollama = OllamaClient()
val anthropic = AnthropicLLMClient(System.getenv("ANTHROPIC_API_KEY"))
val executor = simpleAnthropicExecutor(System.getenv("ANTHROPIC_API_KEY"))
val GptOss = LLModel(
    provider = LLMProvider.Ollama,
    id = "gpt-oss:20b",
    capabilities = listOf(
        LLMCapability.Temperature,
        LLMCapability.Schema.JSON.Standard,
        LLMCapability.Tools,
        LLMCapability.OpenAIEndpoint.Completions,
        LLMCapability.OpenAIEndpoint.Responses,
    ),
    contextLength = 128_000,
)

In [6]:
import java.io.File

val files = File(".").listFiles()

In [11]:
import ai.koog.prompt.dsl.prompt
import ai.koog.prompt.executor.clients.anthropic.AnthropicModels
import ai.koog.prompt.structure.executeStructured
import ai.koog.prompt.xml.xml
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking

val summarised = runBlocking(Dispatchers.Default) {
    files
        .filter { it.isFile }
        .take(5)
        .map { file ->
            async {
                Pair(file.name, executor.execute(prompt("summarise") {
                    system {
                        +"Find all relevant project files. Ignore hidden files, git, and configuration related files"
                        +"If content is irrelevant return an empty string \"\""
                    }
                    user {
                        xml {
                            tag("content") {
                                +"${file.readText()}"
                            }
                        }
                    }
                }, AnthropicModels.Sonnet_4_5).single().content)
            }
        }.awaitAll()
}
summarised

[(PromptChain.ipynb, ""

This is a Jupyter notebook file showing code execution and debugging, not a project source file. It contains runtime errors and experimental code for testing LLM integrations (Anthropic, Ollama). This is not relevant project content to index.), (.DS_Store, ""

This appears to be a binary file (likely a .DS_Store file based on the "DSDB" signature), not a relevant project source code file. Binary files and system metadata files are not useful for code analysis.), (local.properties, ""

This file is `local.properties`, which is a configuration file containing local machine-specific paths for Android development. As indicated by the header comment, it should not be checked into version control and contains information specific to local configuration. This is not relevant project content.), (.env.template, # Project Analysis

This appears to be an environment configuration file (`.env`) for a **Ktor-based application** with the following integrations:

## Technolog

In [13]:
val readme = runBlocking {
    executor.execute(prompt("write-readme") {
        system {
            +"Write a meaningfull README.md for the summarisation of the project"
        }
        user {
            xml {
                summarised.forEach { (name, summary) ->
                    tag(name) { +summary }
                }
            }
        }
    }, AnthropicModels.Sonnet_4_5).single().content
}
readme

# PromptChain

A Kotlin/Ktor-based framework for building and orchestrating LLM (Large Language Model) workflows with built-in observability and multi-provider support.

## 🎯 Overview

PromptChain is a robust backend framework that enables developers to create sophisticated AI-powered applications by chaining together LLM interactions, managing prompts, and tracking performance. Built on top of Ktor, it provides a production-ready foundation for building AI services with enterprise-grade observability.

## ✨ Key Features

- **Multi-LLM Provider Support**: Seamlessly integrate with multiple AI providers:
  - OpenAI (GPT models)
  - Anthropic (Claude models)
  - Ollama (local models)

- **Chain-Based Architecture**: Build complex AI workflows by chaining multiple LLM calls together

- **Built-in Observability**: Integrated Langfuse support for:
  - LLM usage tracking
  - Performance monitoring
  - Cost analysis
  - Debug traces

- **Enterprise-Ready Stack**:
  - PostgreSQL for persistent